In [7]:
import pandas as pd
import datetime

# Wczytanie pliku wejściowego
input_file = 'dane.xlsx'
df = pd.read_excel(input_file)

# Jeśli kolumna 'Kwota' zawiera przecinek jako separator dziesiętny, zamieniamy go na kropkę i konwertujemy na float
df['Kwota'] = df['Kwota'].astype(str).str.replace(',', '.').astype(float)

# Konwersja rezerwacji do typu int (zakładamy, że kolumna 'Res' zawiera dane numeryczne)
df['Res'] = df['Res'].astype(int)

# Pobranie bieżącej daty w wymaganych formatach
today = datetime.date.today()
date_str = today.strftime('%Y-%m-%d')      # Format: 2025-02-20
doc_date_str = today.strftime('%d/%m/%y')    # Format: 20/02/25

# Lista na wiersze wynikowe
output_rows = []

# Liczniki:
# row_counter – do określenia numeru wiersza (mnożymy przez 10000)
# doc_counter – numer dokumentu, zwiększany o 1 dla każdego wiersza
row_counter = 0
doc_counter = 1

for idx, row in df.iterrows():
    nr_konta = row['Nr nab']       # numer konta
    reservation = int(row['Res'])   # numer rezerwacji jako int
    # Zaokrąglamy kwotę do 2 miejsc po przecinku
    amount = round(row['Kwota'], 2)
    
    # Numer dokumentu wg wzoru: "R_GROSZ {doc_date_str} {doc_counter:03d}"
    nr_dokumentu = f"R_GROSZ {doc_date_str} {doc_counter:03d}"
    
    # Opis wg wzoru, kwota formatowana do 2 miejsc po przecinku
    opis = f"Różnice groszowe: {amount:.2f} z rez {reservation}"
    
    # Określenie Nr konta przeciwst. zależnie od wartości kwoty
    if amount > 0:
        konto_przeciwst = "764-99"
    elif amount < 0:
        konto_przeciwst = "763-99"
    else:
        konto_przeciwst = ""
    
    row_counter += 1
    output_rows.append({
        "Nr wiersza": row_counter * 10000,
        "Nazwa instancji dziennika": "ROZ_GROSZ",
        "Nazwa szablonu dziennika": "OGÓLNE",
        "Data księgowania": date_str,
        "Data VAT": date_str,
        "Data dokumentu": date_str,
        "Nr dokumentu": nr_dokumentu,
        "Typ konta": "Nabywca",
        "Nr konta": nr_konta,
        "Reservation No.": reservation,
        "Opis": opis,
        "Kwota": -amount,
        "Kwota Debet": "",
        "Kwota Kredyt": "",
        "Typ konta przeciwst.": "Konto K/G",
        "Nr konta przeciwst.": konto_przeciwst
    })
    
    doc_counter += 1

# Utworzenie DataFrame z wynikowymi danymi
output_df = pd.DataFrame(output_rows)

# Zapis do pliku Excel
output_file = 'result.xlsx'
output_df.to_excel(output_file, index=False)

print(f"Plik {output_file} został wygenerowany.")


Plik result.xlsx został wygenerowany.
